## Preprocessing and Feature Engineering

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
 
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_context("paper", font_scale=1.3)
sns.set_style('white')

# Ignore harmless warnings
import warnings
warnings.filterwarnings("ignore")

import time
from datetime import datetime
import calendar, time

# Lines below are just to ignore warnings
import warnings
warnings.filterwarnings('ignore')

### Loading Power Demand Data

In [2]:
df = pd.read_csv('../dataset/demand_dataframe.csv')[['Time' ,'COA']]
df.head()

,Time,COA
0,01-01-12 1:00,5986.0
1,01-01-12 2:00,5547.0
2,01-01-12 3:00,5233.0
3,01-01-12 4:00,5150.0
4,01-01-12 5:00,5235.0


In [3]:
df[df.Time=='01-08-15 0:00']

,Time,COA
31391,01-08-15 0:00,15277.0


### Loading Weather Data

In [4]:
df_temp = pd.read_csv('../dataset/df_weather2012-2018.csv')[['time' , 'temperature','dewPoint','humidity','windSpeed']]

#### Changing the unix time to normal time

In [5]:
# Changing the unix time to normal time
df_temp['Time']=df_temp.time.map(lambda x: time.strftime("%Y-%m-%d %H:%M:%S", time.localtime(x)))

In [6]:
# Creating an empty Dataset with hourly time to be used for merging the two datasets
df_test = pd.DataFrame(pd.date_range('2012-01-01 01:00:00' , '2017-12-31 23:00:00' , freq='1H') , columns=['Time'])

In [7]:
# Changing the freqency of the index to hour
df_test.index.freq = 'H'

In [8]:
df_test.index

RangeIndex(start=0, stop=52607, step=1)

In [9]:
df_test.head()

,Time
0,2012-01-01 01:00:00
1,2012-01-01 02:00:00
2,2012-01-01 03:00:00
3,2012-01-01 04:00:00
4,2012-01-01 05:00:00


In [10]:
df_temp.head()

,time,temperature,dewPoint,humidity,windSpeed,Time
0,1325365200,14.98,0.01,0.36,4.12,2012-01-01 00:00:00
1,1325368800,13.98,0.01,0.38,3.09,2012-01-01 01:00:00
2,1325372400,11.98,0.99,0.47,2.06,2012-01-01 02:00:00
3,1325376000,11.98,0.99,0.47,2.06,2012-01-01 03:00:00
4,1325379600,11.98,0.01,0.44,2.06,2012-01-01 04:00:00


In [11]:
# droping the time column
df_temp.drop(columns=['time'] , inplace=True)

In [12]:
# setting the index to be the time
df = df.set_index('Time')
df_temp = df_temp.set_index('Time')
df_test = df_test.set_index('Time')

In [13]:
# Changing the type of the index to datetime format
df.index = pd.to_datetime(df.index, format="%d-%m-%y %H:%M" , dayfirst=True)
df_temp.index = pd.to_datetime(df_temp.index)
df_test.index = pd.to_datetime(df_test.index)

In [14]:
df.tail()

,COA
Time,
2017-12-31 19:00:00,9495.0
2017-12-31 20:00:00,9365.0
2017-12-31 21:00:00,9250.0
2017-12-31 22:00:00,8931.0
2017-12-31 23:00:00,8679.0


In [15]:
df_temp.head()

,temperature,dewPoint,humidity,windSpeed
Time,,,,
2012-01-01 00:00:00,14.98,0.01,0.36,4.12
2012-01-01 01:00:00,13.98,0.01,0.38,3.09
2012-01-01 02:00:00,11.98,0.99,0.47,2.06
2012-01-01 03:00:00,11.98,0.99,0.47,2.06
2012-01-01 04:00:00,11.98,0.01,0.44,2.06


In [16]:
df_temp.tail()

,temperature,dewPoint,humidity,windSpeed
Time,,,,
2018-12-31 19:00:00,16.65,8.59,0.59,3.88
2018-12-31 20:00:00,14.73,10.03,0.73,3.45
2018-12-31 21:00:00,14.76,10.18,0.74,3.49
2018-12-31 22:00:00,13.81,9.32,0.74,2.40
2018-12-31 23:00:00,12.83,9.23,0.79,2.28


In [17]:
df_t = df_temp['2012-01-01 01:00:00':'2017-12-31 23:00:00'].merge(df_test , how = 'outer' ,on = 'Time' ).sort_values('Time')

In [18]:
df_l = df['2012-01-01 01:00:00':'2017-12-31 23:00:00'].merge(df_test , how = 'outer' ,on = 'Time' ).sort_values('Time')

In [19]:
len(df_l)

52607

In [20]:
len(df_t)

52607

In [21]:
# Checking if there is any null data
df_t.isnull().sum()

temperature    77
dewPoint       77
humidity       77
windSpeed      81
dtype: int64

In [22]:
# Doing a forword fill for empty data
df_t.fillna(method='ffill' , inplace=True)

In [23]:
df_t.isnull().sum()

temperature    0
dewPoint       0
humidity       0
windSpeed      0
dtype: int64

In [24]:
df_l.index.freq = '1H'

In [25]:
df_t.index.freq = '1H'

## Feature Engineering

In [26]:
# collecting the main weather data to the final dataframe
df_l['temp'] = df_t.temperature.values
df_l['dewPoint'] = df_t.dewPoint.values
df_l['humidity'] = df_t.humidity.values
df_l['windSpeed'] = df_t.windSpeed.values

In [27]:
df_l = df_l.reset_index()

In [28]:
df_l.dtypes

Time         datetime64[ns]
COA                 float64
temp                float64
dewPoint            float64
humidity            float64
windSpeed           float64
dtype: object

In [29]:
# Getting more features from the datetime
df_l['year'] = df_l['Time'].apply(lambda x: x.year)
df_l['quarter'] = df_l['Time'].apply(lambda x: x.quarter)
df_l['month'] = df_l['Time'].apply(lambda x: x.month)
df_l['day'] = df_l['Time'].apply(lambda x: x.day)
df_l.sort_values('Time', inplace=True, ascending=True)
df_l["weekday"]=df_l.apply(lambda row: row["Time"].weekday(),axis=1)
df_l["weekday"] = df_l["weekday"].astype(str)
df_l = pd.get_dummies(df_l)

In [30]:
df_l = df_l.set_index('Time',drop=False)

In [31]:
#%A - Adding full weekday name
df_l['weekday_name']=df_l.index.strftime("%A")


In [32]:
# Adding the hour and the weekday number
df_l['hour_24']=df_l.index.strftime("%H")
df_l['weekday_no']=df_l.index.map(lambda x:x.weekday())


In [34]:
df_l.head(5)

,Time,COA,temp,dewPoint,humidity,windSpeed,year,quarter,month,day,weekday_0,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_name,hour_24,weekday_no
Time,,,,,,,,,,,,,,,,,,,,
2012-01-01 01:00:00,2012-01-01 01:00:00,5986.0,13.98,0.01,0.38,3.09,2012,1,1,1,0,0,0,0,0,0,1,Sunday,01,6
2012-01-01 02:00:00,2012-01-01 02:00:00,5547.0,11.98,0.99,0.47,2.06,2012,1,1,1,0,0,0,0,0,0,1,Sunday,02,6
2012-01-01 03:00:00,2012-01-01 03:00:00,5233.0,11.98,0.99,0.47,2.06,2012,1,1,1,0,0,0,0,0,0,1,Sunday,03,6
2012-01-01 04:00:00,2012-01-01 04:00:00,5150.0,11.98,0.01,0.44,2.06,2012,1,1,1,0,0,0,0,0,0,1,Sunday,04,6
2012-01-01 05:00:00,2012-01-01 05:00:00,5235.0,11.98,0.01,0.44,3.60,2012,1,1,1,0,0,0,0,0,0,1,Sunday,05,6


In [36]:
# Define an empty column in the df_l dataframe
df_l['weekend_bool'] = pd.Series()

In [37]:
# Setting up the weekend boolean for the Thursday & Friday weekend (Before 29-6-2013)
df_l['weekend_bool'][df_l.Time <='2013-06-29']=df_l.weekday_no.apply(lambda x: int(1) if x==3 or x==4 else int(0))

# Setting up the weekend boolean for the Friday & Saterday weekend (After 29-6-2013)
df_l['weekend_bool'][df_l.Time >='2013-06-29']=df_l.weekday_no.apply(lambda x: int(1) if x==4 or x==5 else int(0))

# # Setting up the weekend_bool type to integer
# df_l.weekend_bool=df_l.weekend_bool.astype('int')

In [47]:
# Creating a new empty feature for schoold vacation
df_l['school_vacation'] = pd.Series()

In [61]:
# Below are the school vacations during the years 2012-2017 to be added to the school vacation feature

df_l['school_vacation'][(df_l.Time >='2012-01-19')&(df_l.Time <'2012-01-28')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2012-03-22')&(df_l.Time <'2012-03-31')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2012-06-07')&(df_l.Time <'2012-09-01')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2012-09-23')&(df_l.Time <'2012-09-24')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2012-10-18')&(df_l.Time <'2012-11-02')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2013-01-17')&(df_l.Time <'2013-01-26')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2013-03-21')&(df_l.Time <'2013-03-30')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2013-06-06')&(df_l.Time <'2013-09-01')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2013-09-23')&(df_l.Time <'2013-09-24')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2013-10-10')&(df_l.Time <'2013-10-21')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2014-01-17')&(df_l.Time <'2014-01-26')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2014-03-21')&(df_l.Time <'2014-03-30')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2014-06-06')&(df_l.Time <'2014-08-31')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2014-09-23')&(df_l.Time <'2014-09-24')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2014-09-30')&(df_l.Time <'2014-10-12')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2015-01-16')&(df_l.Time <'2015-01-25')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2015-03-20')&(df_l.Time <'2015-03-29')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2015-06-05')&(df_l.Time <'2015-08-23')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2015-09-18')&(df_l.Time <'2015-10-04')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2016-01-08')&(df_l.Time <'2016-01-17')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2016-03-11')&(df_l.Time <'2016-03-20')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2016-05-26')&(df_l.Time <'2016-09-18')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2016-09-22')&(df_l.Time <'2016-09-23')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2016-11-09')&(df_l.Time <'2016-11-20')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2017-01-27')&(df_l.Time <'2017-02-05')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2017-03-29')&(df_l.Time <'2017-04-09')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2017-06-16')&(df_l.Time <'2017-09-17')]=df_l.school_vacation.apply(lambda x: 1)
df_l['school_vacation'][(df_l.Time >='2017-09-24')&(df_l.Time <'2017-09-25')]=df_l.school_vacation.apply(lambda x: 1)


In [68]:
# filling empty school vacation days with 0
df_l.school_vacation.fillna(0,inplace=True)

In [70]:
# Showing different values for school vacation
df_l.school_vacation.value_counts()

0.0    35183
1.0    17424
Name: school_vacation, dtype: int64

In [38]:
df_l.reset_index(drop=True,inplace=True)

In [41]:
df_l

,Time,COA,temp,dewPoint,humidity,windSpeed,year,quarter,month,day,...,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_name,hour_24,weekday_no,weekend_bool
0,2012-01-01 01:00:00,5986.0,13.98,0.01,0.38,3.09,2012,1,1,1,...,0,0,0,0,0,1,Sunday,01,6,0.0
1,2012-01-01 02:00:00,5547.0,11.98,0.99,0.47,2.06,2012,1,1,1,...,0,0,0,0,0,1,Sunday,02,6,0.0
2,2012-01-01 03:00:00,5233.0,11.98,0.99,0.47,2.06,2012,1,1,1,...,0,0,0,0,0,1,Sunday,03,6,0.0
3,2012-01-01 04:00:00,5150.0,11.98,0.01,0.44,2.06,2012,1,1,1,...,0,0,0,0,0,1,Sunday,04,6,0.0
4,2012-01-01 05:00:00,5235.0,11.98,0.01,0.44,3.60,2012,1,1,1,...,0,0,0,0,0,1,Sunday,05,6,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52602,2017-12-31 19:00:00,9495.0,18.98,-2.00,0.24,1.50,2017,4,12,31,...,0,0,0,0,0,1,Sunday,19,6,0.0
52603,2017-12-31 20:00:00,9365.0,16.98,-2.00,0.27,1.50,2017,4,12,31,...,0,0,0,0,0,1,Sunday,20,6,0.0
52604,2017-12-31 21:00:00,9250.0,15.98,-2.00,0.29,1.50,2017,4,12,31,...,0,0,0,0,0,1,Sunday,21,6,0.0
52605,2017-12-31 22:00:00,8931.0,14.98,-2.00,0.31,1.00,2017,4,12,31,...,0,0,0,0,0,1,Sunday,22,6,0.0


In [42]:
# check the weekend_bool values
df_l[(df_l.weekday_no==5)]

,Time,COA,temp,dewPoint,humidity,windSpeed,year,quarter,month,day,...,weekday_1,weekday_2,weekday_3,weekday_4,weekday_5,weekday_6,weekday_name,hour_24,weekday_no,weekend_bool
143,2012-01-07 00:00:00,6058.000000,12.98,4.01,0.54,1.02,2012,1,1,7,...,0,0,0,0,1,0,Saturday,00,5,0.0
144,2012-01-07 01:00:00,5491.000000,12.98,4.01,0.54,1.02,2012,1,1,7,...,0,0,0,0,1,0,Saturday,01,5,0.0
145,2012-01-07 02:00:00,5158.000000,11.98,4.01,0.58,1.55,2012,1,1,7,...,0,0,0,0,1,0,Saturday,02,5,0.0
146,2012-01-07 03:00:00,4987.000000,11.98,0.99,0.47,2.57,2012,1,1,7,...,0,0,0,0,1,0,Saturday,03,5,0.0
147,2012-01-07 04:00:00,4917.000000,11.98,0.99,0.47,1.55,2012,1,1,7,...,0,0,0,0,1,0,Saturday,04,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52578,2017-12-30 19:00:00,9439.000000,17.98,-2.99,0.24,1.50,2017,4,12,30,...,0,0,0,0,1,0,Saturday,19,5,1.0
52579,2017-12-30 20:00:00,9438.000000,15.98,-1.01,0.31,2.09,2017,4,12,30,...,0,0,0,0,1,0,Saturday,20,5,1.0
52580,2017-12-30 21:00:00,9251.000000,15.38,3.19,0.44,0.89,2017,4,12,30,...,0,0,0,0,1,0,Saturday,21,5,1.0
52581,2017-12-30 22:00:00,9116.182617,14.88,2.77,0.44,0.89,2017,4,12,30,...,0,0,0,0,1,0,Saturday,22,5,1.0


In [43]:
df_l.dtypes

Time            datetime64[ns]
COA                    float64
temp                   float64
dewPoint               float64
humidity               float64
windSpeed              float64
year                     int64
quarter                  int64
month                    int64
day                      int64
weekday_0                uint8
weekday_1                uint8
weekday_2                uint8
weekday_3                uint8
weekday_4                uint8
weekday_5                uint8
weekday_6                uint8
weekday_name            object
hour_24                 object
weekday_no               int64
weekend_bool           float64
dtype: object

In [73]:
# Exporting the data to a csv file
df_l.to_csv('../dataset/full_COA_v3.csv' , index=False)